In [2]:
# Project idea -> add workout counter in look studio google
# Packages required for this project:
import requests
import pandas as pd
import numpy as np
import pygsheets
import datetime
from datetime import date
import json
import webbrowser
from time import sleep
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import sys
sys.path.append(r'C:\\Users\\Manuel Elizaldi\\Desktop\\Learning-Testing\\PyStrava\\Scripts')
from Functions import *

In [14]:
# Setting up parameters for gspread - updating google sheet
service_file_path = r'C:\Users\Manuel Elizaldi\Desktop\Learning-Testing\PyStrava\Credentials\pacific-castle-303123-909a5ddcda92.json'
spreadsheet_id = '1pomkAzlndHBl_czERrwKkoZFUkJRGFjyhRTeoWA6CS4'

# My scope determines what we want to do with the google api
myscope = ['https://spreadsheets.google.com/feeds', 
            'https://www.googleapis.com/auth/drive']

# Authentication
mycred = ServiceAccountCredentials.from_json_keyfile_name(service_file_path,myscope) # type: ignore
client = gspread.authorize(mycred)

# Opening and reading the All_Workouts_Table <- this will contain our not updated list of workouts
mysheet = client.open('workout-data').sheet1
list_of_row = mysheet.get_all_records()

# Parsing json into a dataframe
all_workouts_df = pd.json_normalize(list_of_row)

In [20]:
all_workouts_df[['start_lat','start_long','end_lat','end_long']] = all_workouts_df[['start_lat','start_long','end_lat','end_long']].replace('no value', np.nan)

In [21]:
all_workouts_df.head(1)

,activity_id,name,start_date,sport_type,distance,workout_time_min,calories,total_elevation_gain,start_lat,start_long,...,average_heartrate_score,max_heartrate_score,avg_lap_time_score,lap_count_score,avg_speed_score,max_speed_score,pace_score,effort_score,effort_score_label,effort_score_rank
0,10697198078,Lunch Pickleball,2024-02-04,Pickleball,1.11,24.12,207.0,0.0,NaN,NaN,...,5,10,20,5,5,15,0,102,Low Effort,2


In [22]:
# Uploading to Google Sheet
sheet_name = 'All_Workouts_Table'
WriteToGsheet(service_file_path,spreadsheet_id,sheet_name,all_workouts_df)